In [26]:
import cv2
from rtmlib import Wholebody, draw_skeleton
import random
import pandas as pd
import os
from tqdm import tqdm
import wget
import json
from itertools import product

In [27]:
device = 'cpu'  # cpu, cuda, mps
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

In [28]:
anns_df = pd.read_csv('../data/coco_diff/single_anns/person_anns.csv')
img_url_list = list(anns_df['img_url'].unique())
random.seed(24) # to use same collection of images
img_sample = random.sample(img_url_list, 2000)

In [29]:
filepath_list = []

In [30]:
img_source = "../imgs"

In [31]:
old_cwd = os.getcwd()
new_cwd = os.path.dirname(old_cwd)+'/imgs'
os.chdir(new_cwd)
for url in tqdm(img_sample):
    try:
        file_name = url.split('/')[-1]
        file_path = os.path.join(new_cwd, file_name)
        filepath_list.append(file_path)
        
        if not os.path.exists(file_path):
            wget.download(url)
    except Exception as e:
        print(f"\nError in download of {url}: {e}")
os.chdir(old_cwd)

100%|██████████| 2000/2000 [00:00<00:00, 54197.33it/s]


In [32]:
def crop_image(image, keep_percent, direction=None):
    """Crop the image based on keep_percent from the left or top."""
    h, w, _ = image.shape
    if direction == "side":
        cropped = image[:, :int(w * keep_percent)]
    elif direction == "top":
        cropped = image[:int(h * keep_percent), :]
    else:
        raise ValueError("Direction must be 'side' or 'top'.")
    return cropped

In [33]:
openpose_skeleton = True 

wholebody = Wholebody(to_openpose=openpose_skeleton,
                      mode='balanced', 
                      backend=backend, device=device)

load C:\Users\fedep\.cache\rtmlib\hub\checkpoints\yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend
load C:\Users\fedep\.cache\rtmlib\hub\checkpoints\rtmw-x_simcc-cocktail13_pt-ucoco_270e-256x192-fbef0d61_20230925.onnx with onnxruntime backend


In [45]:
percentages = [0.25, 0.50, 0.75]
parts = ["top", "side"]
keypoints_dict = {f"{part}_{int(percent*100)}": [] for part, percent in product(parts, percentages) }
for filename in tqdm(list(os.listdir(img_source))):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(img_source, filename)  
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error in image loading: {filename}")
            continue

        crops = {}

        for percent in percentages:
            crops[f"side_{int(percent * 100)}"] = crop_image(image, percent, "side")
            crops[f"top_{int(percent * 100)}"] = crop_image(image, percent, "top")

        img_id = filename.split('\\')[-1].strip('.jpg').lstrip('0')

        for crop_type, cropped_image in (crops.items()):
            keypoints, scores = wholebody(cropped_image)
            frame_obj = {'Img_ID': f"{img_id}", 'people': []}
            for index, (person_kp, person_score) in enumerate(zip(keypoints, scores)):
                person_keypoints = [{"x": float(kp[0]), "y": float(kp[1]), "score": float(sc)} for kp, sc in zip(person_kp, person_score)]
                frame_obj["people"].append(person_keypoints)
            keypoints_dict[f"{crop_type}"].append(frame_obj)

100%|██████████| 401/401 [57:24<00:00,  8.59s/it]  


In [46]:
with open("./COCO/keypoints_cropped.json", "w") as f:
    json.dump(keypoints_dict, f, indent=4)